In [1]:
library(tidyverse)
source("../global_variables.R")
source("../../../cfrna_template/theme_ggplot_cfrna.R")

library(extrafont)
# fonts()

library(readxl)


meta = read.csv("./metadata_all.csv")
meta %>% head()


FILL_KEY = INFLAMCAT_FILL_KEY


FILL_KEY_ORIGIN = c("CNH" = "#E5002B",
                    "EMORY" = "#EDC967", #"#b58500",
                    "UCSD" = "#00629B",
                    "UCSF" = "#9DC183" #FA8128
                    )

# FILL_KEY_ORIGIN = c("CNH" = "#9467bd",
#                     "EMORY" = "#F7EF8A",
#                     "UCSD" = "#1f77b4",
#                     "UCSF" = "#ff7f0e")

                    

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.4     ✔ purrr   1.0.2
✔ tibble  3.2.1     ✔ dplyr   1.1.4
✔ tidyr   1.3.0     ✔ stringr 1.5.1
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Registering fonts with R



,PTID,sample_id,diagnosis,origin,severity,date_dif,age,inflam_cat,Xsample_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
1,153000,cfrna_kd_202_trimmed,KD,UCSD,KD subtype 3,0,4.744907,KD,cfrna_kd_202_trimmed
2,153002,cfrna_kd_128,KD,UCSD,KD subtype 1,0,8.800000,KD,cfrna_kd_128
3,153027,cfrna_kd_121,KD,UCSD,KD subtype 4,0,2.900000,KD,cfrna_kd_121
4,153034,cfrna_kd_47,KD,UCSD,KD subtype 4,0,2.000000,KD,cfrna_kd_47
5,153036,cfrna_kd_207_trimmed,KD,UCSD,KD subtype 2,0,1.741309,KD,cfrna_kd_207_trimmed
6,153047,cfrna_kd_218_trimmed,KD,UCSD,KD subtype 2,0,2.428409,KD,cfrna_kd_218_trimmed


---
## Figure 1

In [6]:
meta %>% nrow
table(meta$inflam_cat) %>% data.frame() %>%
    rename(group = Var1) %>%
    arrange(desc(Freq))

[1] 370

group,Freq
<fct>,<int>
viral_infection,105
KD,101
MISC,97
bacterial_infection,41
Healthy,17
other,9


In [7]:
#######################
### SAMPLE BAR CHART
#######################

counts = meta %>% 
    group_by(origin, inflam_cat) %>% 
    summarize(num = n()) %>% 
    mutate(origin = factor(origin, levels = rev(c("UCSD","EMORY","CNH","UCSF")))) %>% 
    mutate(inflam_cat = factor(inflam_cat, levels = names(INFLAMCAT_FILL_KEY))) %>% 
    group_by(inflam_cat) %>% 
    mutate(csum = rev(cumsum(rev(num))))

# counts$ymin <- c(0, head(counts$ymax,n=-1))

samp_cnts_lng <- counts %>% 
    ggplot(aes(x=inflam_cat, y=num, fill=origin))+
    geom_bar(position="stack", stat="identity", color = "black", size = 0.25, width =.65)+
    labs(y="Sample Count")+
    # geom_label(aes(x = inflam_cat, y = csum, label = num), 
    #                     size = 2,  show.legend = FALSE, nudge_x = .3, nudge_y = -5)+
    theme_bw() + 
    labs(x="")+
    theme_cfrna_print()+
    theme(text = element_text(color = "black"),
        axis.text = element_text(size = 6),
        # axis.text.x = element_text(size = 8, angle = 45, hjust = 1),
        # axis.title.x = element_blank(),
        axis.title.y = element_text(size=8)) +
    scale_y_continuous(breaks = seq(0, 100, 20), limits = c(0, 110))+
    scale_fill_manual(values = FILL_KEY_ORIGIN)

WIDTH= 3.42
HEIGHT= 2.25
pdf(file="figure_plots/OVERVIEW_sample_counts.LONG.pdf",
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
samp_cnts_lng + theme(legend.position = "none")
dev.off()

`summarise()` has grouped output by 'origin'. You can override using the
`.groups` argument.


Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”


png 
  2

In [4]:
#######################
### PIE CHART (INSET)
#######################

library(ggrepel)

all_origin_pie <- meta %>% 
    group_by(origin) %>% 
    dplyr::count() %>% 
    mutate(origin = factor(origin, levels = c("UCSD","EMORY","CNH","UCSF"))) %>% 
    ungroup() %>% 
    group_by(origin) %>% 
    mutate(frac = n/sum(n)) %>% 
    arrange(origin)

df2 <- all_origin_pie %>% 
    mutate(csum = rev(cumsum(rev(n))), 
    pos = n/2 + lead(csum, 1),
    pos = if_else(is.na(pos), n/2, pos)) %>% 
    arrange(csum)

df2$ymax <- c(df2$csum)
df2$ymin <- c(0, head(df2$ymax,n=-1))
df2$labelPosition <- (df2$ymax + df2$ymin)/2
df2$label = paste0("n=",as.character(df2$n))

allsamp_plt <- df2 %>% 
    ggplot(aes(ymax=ymax, ymin=ymin, xmax=4, xmin=3, fill=origin)) +
        geom_rect(color = "black", size = .25) +
        geom_label_repel(x = 4, aes(y = labelPosition, label = label), 
                        size = 2,  show.legend = FALSE)+ #nudge_x = 1,
        scale_fill_brewer(palette=3) +
        scale_color_brewer(palette=3) +
        coord_polar(theta="y") +
        xlim(c(1, 4)) +
        theme_void() +
        theme(legend.position = "none")+
        scale_fill_manual(values = FILL_KEY_ORIGIN)+
        scale_color_manual(values = FILL_KEY_ORIGIN)

allsamp_plt <- allsamp_plt #+ geom_text(x=1,y = max(df2$n)*0.25, label = "", size = 3, family = "Droid Sans")

WIDTH= 1
HEIGHT= 1

pdf(file="figure_plots/OVERVIEW_origin_pie.inset.pdf",
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
allsamp_plt
dev.off()

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.


png 
  2

In [5]:
#######################
### AGE BOX CHART
#######################

set.seed(42)

new_names <- c("MIS-C","KD","Viral Inf.", "Bact. Inf.", "Other", "Healthy")
### age, races, severe vs moderate (ICU or not), CRP
age_box <- meta %>% 
    mutate(inflam_cat = factor(inflam_cat, levels = names(INFLAMCAT_FILL_KEY))) %>%
    mutate(inflam_cat_new = case_when(
            inflam_cat == "MISC" ~ "MIS-C",
            inflam_cat == "KD" ~ "KD",
            grepl("viral",inflam_cat) ~ "Viral Inf.",
            grepl("bacterial",inflam_cat) ~ "Bact. Inf.",
            inflam_cat == "other" ~ "Other",
            inflam_cat == "Healthy" ~ "Healthy",
                TRUE ~ "Other"
    )) %>% 
    mutate(inflam_cat_new = factor(inflam_cat_new, levels = new_names)) %>%
    ggplot(aes(x=inflam_cat_new, y=age))+
    geom_boxplot(outlier.shape = NA, color = "black", alpha = 0.75)+
    geom_jitter(width = 0.2, height = 0, color = "black", size = .75, alpha = 0.5, stroke=NA)+
    theme_cfrna_print()+
    labs(y="Age")+
    theme(axis.title.x = element_blank())+
    scale_x_discrete(guide = guide_axis(n.dodge = 2))

WIDTH= 3.3 / 2
HEIGHT= 1.3

pdf(file="figure_plots/OVERVIEW_age.box.pdf",
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
age_box
dev.off()

png 
  2

In [7]:
#######################
### ICU BAR CHART
#######################
tab1 <- read.csv("./table1_dataframe.csv")

new_names <- c("MIS-C","KD","Viral Inf.", "Bact. Inf.", "Other")



icu_box <- tab1 %>% 

        ### set up x axis
    mutate(inflam_cat = factor(inflam_cat, levels = names(INFLAMCAT_FILL_KEY))) %>%
    filter(inflam_cat != "Healthy") %>%
    mutate(inflam_cat_new = case_when(
            inflam_cat == "MISC" ~ "MIS-C",
            inflam_cat == "KD" ~ "KD",
            grepl("viral",inflam_cat) ~ "Viral Inf.",
            grepl("bacterial",inflam_cat) ~ "Bact. Inf.",
            inflam_cat == "other" ~ "Other",
            inflam_cat == "Healthy" ~ "Healthy",
                TRUE ~ "Other"
    )) %>% 
    mutate(inflam_cat_new = factor(inflam_cat_new, levels = new_names)) %>%

        #### count groups
    filter(!is.na(icu_yes)) %>%
    group_by(inflam_cat_new, icu_yes) %>%
    summarize(n = n()) %>%
    mutate(icu_yes = factor(icu_yes, levels = rev(c(1,0)))) %>%

        ### plot

    ggplot(aes(x=inflam_cat_new, y=n, fill = factor(icu_yes)))+
        geom_bar(position="fill", stat="identity", color = "black", size = 0.25, width =.65)+
        theme_cfrna_print()+
        labs(y = "Fraction")+
        theme(axis.title.x = element_blank())+
        scale_x_discrete(guide = guide_axis(n.dodge = 2))+
        scale_fill_manual(values = c("1" = "grey", "0" = "white"))

WIDTH= (3.3 / 2)  * 0.97
HEIGHT= 1.3

pdf(file="figure_plots/OVERVIEW_ICU.bar.pdf",
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
icu_box
dev.off()


`summarise()` has grouped output by 'inflam_cat_new'. You can override using
the `.groups` argument.


png 
  2

In [8]:
#######################
### SEX BAR CHART
#######################
tab1 <- read.csv("./table1_dataframe.csv")

new_names <- c("MIS-C","KD","Viral Inf.", "Bact. Inf.", "Other", "Healthy")



sex_box <- tab1 %>% 

        ### set up x axis
    mutate(inflam_cat = factor(inflam_cat, levels = names(INFLAMCAT_FILL_KEY))) %>%
    mutate(inflam_cat_new = case_when(
            inflam_cat == "MISC" ~ "MIS-C",
            inflam_cat == "KD" ~ "KD",
            grepl("viral",inflam_cat) ~ "Viral Inf.",
            grepl("bacterial",inflam_cat) ~ "Bact. Inf.",
            inflam_cat == "other" ~ "Other",
            inflam_cat == "Healthy" ~ "Healthy",
                TRUE ~ "Other"
    )) %>% 
    mutate(inflam_cat_new = factor(inflam_cat_new, levels = new_names)) %>%

        #### count groups
    filter(!is.na(sex_male)) %>%
    group_by(inflam_cat_new, sex_male) %>%
    summarize(n = n()) %>%
    mutate(sex_male = factor(sex_male, levels = rev(c(1,0)))) %>%

        ### plot

    ggplot(aes(x=inflam_cat_new, y=n, fill = factor(sex_male)))+
        geom_bar(position="fill", stat="identity", color = "black", size = 0.25, width =.65)+
        theme_cfrna_print()+
        labs(y = "Fraction")+
        theme(axis.title.x = element_blank())+
        scale_x_discrete(guide = guide_axis(n.dodge = 2))+
        scale_fill_manual(values = c("1" = "grey", "0" = "white"))

WIDTH= (3.3 / 2) * 1.05
HEIGHT= 1.3

pdf(file="figure_plots/OVERVIEW_SEX.bar.pdf",
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
sex_box
dev.off()


`summarise()` has grouped output by 'inflam_cat_new'. You can override using
the `.groups` argument.


png 
  2

In [46]:
#######################
### CRP BOX CHART
#######################

library(lubridate)
tab1 <- read.csv("./table1_dataframe.csv")
non_ucsd <- tab1 %>% filter(origin != "UCSD") %>% pull(PTID)

redcap = read.csv("../../meta_data/cfrna/RedCap-df_merged.csv")
redcap <- redcap %>% filter(PTID %in% non_ucsd) %>% select(PTID, date_hosp, crp) %>% 
    filter(!is.na(crp)) %>% mutate(date_hosp = ymd(date_hosp)) %>% 
    group_by(PTID) %>% slice_min(date_hosp, n=1) %>% slice_max(crp, n=1) %>% rename(crp_new = crp)

tab1_crp <- merge(tab1, redcap, by = "PTID", all.x = TRUE) %>% 
    mutate(crp = ifelse(is.na(crp), crp_new, crp))


new_names <- c("MIS-C","KD","Viral Inf.", "Bact. Inf.", "Other")


crp_box <- tab1_crp %>% 
    mutate(inflam_cat = factor(inflam_cat, levels = names(INFLAMCAT_FILL_KEY))) %>%
    filter(inflam_cat != "Healthy") %>%
    mutate(inflam_cat_new = case_when(
            inflam_cat == "MISC" ~ "MIS-C",
            inflam_cat == "KD" ~ "KD",
            grepl("viral",inflam_cat) ~ "Viral Inf.",
            grepl("bacterial",inflam_cat) ~ "Bact. Inf.",
            inflam_cat == "other" ~ "Other",
            inflam_cat == "Healthy" ~ "Healthy",
                TRUE ~ "Other"
    )) %>% 
    mutate(inflam_cat_new = factor(inflam_cat_new, levels = new_names)) %>%
    ggplot(aes(x=inflam_cat_new, y=crp))+
    geom_boxplot(outlier.shape = NA, color = "black", alpha = 0.75)+
    geom_jitter(width = 0.2, height = 0, color = "black", size = .75, alpha = 0.5, stroke=NA)+
    theme_cfrna_print()+
    labs(y = "CRP (mg/dL)")+
    theme(axis.title.x = element_blank())+
    coord_cartesian(ylim = c(0, 75))+
    scale_x_discrete(guide = guide_axis(n.dodge = 2))

WIDTH= (3.3 / 2)  * .93
HEIGHT= 1.3

pdf(file="figure_plots/OVERVIEW_CRP.box.pdf",
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
crp_box
dev.off()


tab1 %>% arrange(desc(crp)) %>% head(5) %>% select(inflam_cat, crp)

Warning message:
“Removed 25 rows containing non-finite values (`stat_boxplot()`).”
Warning message:
“Removed 25 rows containing missing values (`geom_point()`).”


png 
  2

,inflam_cat,crp
,<chr>,<dbl>
1,bacterial_infection,225.0
2,bacterial_infection,103.1
3,bacterial_infection,55.5
4,MISC,45.3
5,MISC,42.7


In [42]:
library(lubridate)
tab1 <- read.csv("./table1_dataframe.csv")
non_ucsd <- tab1 %>% filter(origin != "UCSD") %>% pull(PTID)

redcap = read.csv("../../meta_data/cfrna/RedCap-df_merged.csv")
redcap <- redcap %>% filter(PTID %in% non_ucsd) %>% select(PTID, date_hosp, crp) %>% 
    filter(!is.na(crp)) %>% mutate(date_hosp = ymd(date_hosp)) %>% 
    group_by(PTID) %>% slice_min(date_hosp, n=1) %>% slice_max(crp, n=1) %>% rename(crp_new = crp)

tab1_crp <- merge(tab1, redcap, by = "PTID", all.x = TRUE) %>% 
    mutate(crp = ifelse(is.na(crp), crp_new, crp))

# date_hosp

---
## Table 1

In [4]:
ucsf_data = read_xlsx("./data/Cornell Manifest Bacterial Sepsis Samples-2 1-3.xlsx") %>% 
    rename(PTID = `Study ID`, samp_id = `Sample ID`, race = Race, eth = Ethnicity, sex = Sex, crp = `CRP levels (at admission)`, icu = `ICU status`, oxygen = `Oxygen requirements (on date of sample collected or within +/- 1 day)`)%>%
    mutate(PTID = samp_id) %>%
    filter(PTID %in% meta$PTID) %>%
    select(PTID, samp_id, race, eth, sex, crp, icu,oxygen) %>%
    mutate(eth_HispanicOrLatino = ifelse(eth == "Hispanic or Latino", 1, 0),
           eth_NotHispanicOrLatino = ifelse(eth == "Not Hispanic or Latino",1,0)) %>% 
    mutate(race_HispanicOrLatino = ifelse(eth == "Hispanic or Latino", 1, 0),
           race_AmericanIndianOrAlaskaNative = ifelse(grepl("American Indian",race),1,0), # & (race_MoreThanOne!=1),1,0),
            race_Asian = ifelse(grepl("Asian",race),1,0), #& (race_MoreThanOne!=1),1,0),
            race_BlackOrAA = ifelse(grepl("Black",race),1,0), #& (race_MoreThanOne!=1),1,0),
            race_NativeHawaiianOrOtherPacificIslander = ifelse(grepl("Native Hawaiian",race),1,0), #& (race_MoreThanOne!=1),1,0),
            race_White = ifelse(grepl("White",race),1,0),
            race_Other = ifelse(grepl("Other",race),1,0)) %>% select(!race) %>%
    mutate(race_MoreThanOne = ifelse((race_HispanicOrLatino + race_AmericanIndianOrAlaskaNative + race_Asian + race_BlackOrAA + race_NativeHawaiianOrOtherPacificIslander + race_White + race_Other)>1,1,0)) %>%
    mutate(race_AmericanIndianOrAlaskaNative = ifelse(race_MoreThanOne == 1, 0, race_AmericanIndianOrAlaskaNative),
            race_Asian = ifelse(race_MoreThanOne == 1, 0, race_Asian),
            race_BlackOrAA = ifelse(race_MoreThanOne == 1, 0, race_BlackOrAA),
            race_NativeHawaiianOrOtherPacificIslander = ifelse(race_MoreThanOne == 1, 0, race_NativeHawaiianOrOtherPacificIslander),
            race_White = ifelse(race_MoreThanOne == 1, 0, race_White),
            race_Other = ifelse(race_MoreThanOne == 1, 0, race_Other),
            race_HispanicOrLatino = ifelse(race_MoreThanOne == 1, 0, race_HispanicOrLatino)) %>% 
    mutate(oxygen_HFNC = ifelse(grepl("HFNC",oxygen),1,0),
           oxygen_LFNC = ifelse(grepl("LFNC",oxygen),1,0),
           oxygen_Ventilator = ifelse(grepl("Ventilator",oxygen),1,0),
           oxygen_None = ifelse(grepl("None",oxygen),1,0),
           oxygen_NasalCannula = ifelse(grepl("Nasal Cannula",oxygen),1,0),
           oxygen_BiPap = ifelse(grepl("BiPap",oxygen),1,0),
           oxygen_CPAP = ifelse(grepl("CPAP",oxygen),1,0),
           oxygen_ECMO = ifelse(grepl("ECMO",oxygen),1,0),
           oxygen_FaceMask = ifelse(grepl("Face Mask",oxygen),1,0)) %>% select(!oxygen) %>% 
    mutate(sex_male = ifelse(grepl("Male",sex),1,0),
                sex_female = ifelse(grepl("Female",sex),1,0)) %>% select(!sex) %>% 
    mutate(icu_yes = ifelse(grepl("Yes",icu),1,0),
           icu_no = ifelse(grepl("No",icu),1,0)) %>% select(!icu) %>% 
    mutate(origin = "UCSF")

# pvail_data = read.delim("/workdir/cjl332/prevail/marker_genes/meta_data/cfrna/3_all_variables.tsv") %>% 
#     select(PTID, sex, crp,  ethnicity___1, ethnicity___0, race___1, race___2, race___3, race___4, race___5, race___6, ressupport)
pvail_data = read.csv("./data/PreVAILCDEs-DemographicsPedInlfl_DATA_2024-05-11_1036.csv") %>% 
    rename(PTID = siteid) %>% 
    mutate(PTID = toupper(PTID)) %>%
    filter(PTID %in% meta$PTID) %>%
    mutate(eth_HispanicOrLatino = ethnicity___1, eth_NotHispanicOrLatino = ethnicity___0) %>% #select(!starts_with("ethnicity__")) %>% 
    mutate(race_MoreThanOne = ifelse( (race___1 + race___2 + race___3 + race___4 + race___5 + race___6 + ethnicity___1) > 1, 1,0 )) %>%
    mutate(race_AmericanIndianOrAlaskaNative = ifelse(race_MoreThanOne == 1, 0, race___1),
            race_Asian = ifelse(race_MoreThanOne == 1, 0, race___2),
            race_BlackOrAA = ifelse(race_MoreThanOne == 1, 0, race___3),
            race_NativeHawaiianOrOtherPacificIslander = ifelse(race_MoreThanOne == 1, 0, race___4),
            race_White = ifelse(race_MoreThanOne == 1, 0, race___5),
            race_Other = ifelse(race_MoreThanOne == 1, 0, race___6),
            race_HispanicOrLatino = ifelse(race_MoreThanOne == 1, 0, ethnicity___1)) %>% select(!starts_with("race__")) %>% select(!starts_with("ethnicity__")) %>% 
    mutate(oxygen_NasalCannula = type_oxygen___1,
            oxygen_FaceMask = type_oxygen___5,
            oxygen_HFNC = type_oxygen___2,
           oxygen_BiPap = type_oxygen___3,
           oxygen_Ventilator = type_oxygen___4,
           oxygen_ECMO = type_oxygen___6,
           oxygen_CPAP = type_oxygen___7,
           oxygen_None = ifelse(sum(type_oxygen___1, type_oxygen___2,type_oxygen___3,type_oxygen___4,type_oxygen___5,type_oxygen___6, type_oxygen___7) == 0, 1, 0))  %>% select(!starts_with("type_oxygen__")) %>% 
    mutate(sex_male = ifelse(sex == 1, 1, 0), sex_female = ifelse(sex == 2, 1,0)) %>% select(!sex)%>%
    mutate(icu_yes = ifelse(icu == 1,1,0), icu_no = ifelse(icu == 0,1,0)) %>% select(!icu) %>% 
    select(!study_id) %>% select(!redcap_repeat_instrument) %>% select(!redcap_repeat_instance) %>% select(!icu_onset) %>% 
    mutate(origin = ifelse(grepl("CNH",PTID),"CNH","EMORY")) 


ucsd_data = read_xlsx("/workdir/cjl332/radysChildrens_ucsd/1_DATA/data/All-metadata_cfRNA samples sent_as_of_120423.xlsx") %>% 
        mutate(PTID = gsub(",","", `ID #`)) %>%
        mutate(PTID = gsub("-CHM-","CHM",PTID)) %>%
        mutate(PTID = ifelse(PTID == "173028.2","173028",PTID)) %>%
        mutate(PTID = ifelse(PTID == "3326.2","3326",PTID)) %>%
        filter(PTID %in% meta$PTID) %>%
        mutate(origin = "UCSD") %>% 
        rename(crp = pcrp) %>%
        mutate(eth_HispanicOrLatino = ifelse(eth == 4, 1, 0),eth_NotHispanicOrLatino = ifelse(eth !=4, 1,0)) %>% 
        mutate(race_HispanicOrLatino = ifelse(eth == 4, 1, 0),
                race_AmericanIndianOrAlaskaNative = ifelse(eth == 7,1,0),
            race_Asian = ifelse(eth == 1,1,0),
            race_BlackOrAA = ifelse(eth == 2,1,0),
            race_NativeHawaiianOrOtherPacificIslander = ifelse(eth == 8,1,0),
            race_White = ifelse(eth == 3,1,0),
            race_Other = ifelse(eth == 9,1,0),
            race_MoreThanOne = ifelse(eth == 6, 1, 0)) %>% 
        mutate(oxygen_HFNC = NA,
           oxygen_LFNC = NA,
           oxygen_Ventilator = NA,
           oxygen_None = NA,
           oxygen_NasalCannula = NA,
           oxygen_BiPap = NA,
           oxygen_CPAP = NA,
           oxygen_ECMO = NA,
           oxygen_FaceMask = NA) %>% 
        mutate(sex_male = ifelse(sex == 1, 1, 0), sex_female = ifelse(sex == 2, 1,0))

ucsd_severity = read_xlsx("./data/Multipaper cohort severity 051324.xlsx") %>% 
        rename(icu =`ICU (1=yes, 0=no)`) %>% 
        mutate(icu_yes = ifelse(icu == 1,1,0), icu_no = ifelse(icu == 0,1,0)) %>% select(!icu) %>%
        select(PTID, icu_yes, icu_no) %>% unique()
ucsd_data <- merge(ucsd_data,ucsd_severity,by="PTID")
# ucsd_severity %>% head()
        

ucsf_data <- ucsf_data %>% select(all_of(pvail_data %>% colnames())) 
ucsd_data = ucsd_data %>% select(all_of(pvail_data %>% colnames()))

COL_order = c('PTID','origin','eth_HispanicOrLatino','eth_NotHispanicOrLatino','race_HispanicOrLatino','race_AmericanIndianOrAlaskaNative','race_Asian','race_BlackOrAA','race_NativeHawaiianOrOtherPacificIslander','race_White','race_Other','race_MoreThanOne','sex_male','sex_female','crp','icu_yes','icu_no','oxygen_NasalCannula','oxygen_FaceMask','oxygen_HFNC','oxygen_BiPap','oxygen_Ventilator','oxygen_ECMO','oxygen_CPAP','oxygen_None')


all_table_data = rbind(ucsf_data, pvail_data, ucsd_data) %>% 
    mutate(origin = factor(origin, levels = c("UCSD","EMORY","CNH","UCSF"))) %>% 
    select(all_of(COL_order))

all_table_data %>% head()

meta_tmp <- meta %>% select(PTID, inflam_cat, age)

all_table_data <- merge(meta_tmp, all_table_data, by="PTID", all=TRUE)


all_table_data %>% write.csv("./table1_dataframe.csv",sep=",", row.names = FALSE)

# ucsf_data %>% head()
# pvail_data %>% head()
# pvail_data %>% colnames()
# ucsd_data %>% head()
# ucsd_data %>% colnames()

New names:
• `` -> `...53`
• `` -> `...54`
• `` -> `...55`


PTID,origin,eth_HispanicOrLatino,eth_NotHispanicOrLatino,race_HispanicOrLatino,race_AmericanIndianOrAlaskaNative,race_Asian,race_BlackOrAA,race_NativeHawaiianOrOtherPacificIslander,race_White,⋯,icu_yes,icu_no,oxygen_NasalCannula,oxygen_FaceMask,oxygen_HFNC,oxygen_BiPap,oxygen_Ventilator,oxygen_ECMO,oxygen_CPAP,oxygen_None
<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SEP0156,UCSF,0,1,0,0,0,0,0,1,⋯,1,0,0,0,0,0,0,0,0,0
SEP0216,UCSF,1,0,0,0,0,0,0,0,⋯,0,1,0,0,0,0,0,0,0,1
SEP0250,UCSF,1,0,0,0,0,0,0,0,⋯,0,1,0,0,0,0,0,0,0,1
SEP0313,UCSF,1,0,0,0,0,0,0,0,⋯,0,1,0,0,0,0,0,0,0,1
SEP0315,UCSF,0,1,0,0,0,0,0,1,⋯,1,0,0,0,0,0,1,0,0,0
SEP0368,UCSF,1,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,0,0,0,0


Warning message in write.csv(., "./table1_dataframe.csv", sep = ",", row.names = FALSE):
“attempt to set 'sep' ignored”


In [5]:
COL_order = c("origin",'race_HispanicOrLatino','race_AmericanIndianOrAlaskaNative','race_Asian','race_BlackOrAA','race_NativeHawaiianOrOtherPacificIslander','race_White','race_Other','race_MoreThanOne','sex_male','sex_female','icu_yes','icu_no','oxygen_NasalCannula','oxygen_FaceMask','oxygen_HFNC','oxygen_BiPap','oxygen_Ventilator','oxygen_ECMO','oxygen_CPAP','oxygen_None')
all_table_data <- all_table_data %>% mutate(inflam_cat = factor(inflam_cat, levels = names(INFLAMCAT_FILL_KEY))) %>% 
    mutate(origin = factor(origin, levels = c("UCSD","EMORY","CNH","UCSF")))

for(col in COL_order){
    print("====================================")
    print(col)
    table(all_table_data[[col]], all_table_data$inflam_cat) %>% print()
}

# 'crp',
all_table_data %>% head()

[1] "===================================="
[1] "origin"
       
        MISC KD viral_infection bacterial_infection other Healthy
  UCSD    41 99              69                  25     0       0
  EMORY   43  1              29                   0     3      17
  CNH     12  1               5                   7     6       0
  UCSF     0  0               0                   9     0       0
[1] "===================================="
[1] "race_HispanicOrLatino"
   
    MISC KD viral_infection bacterial_infection other Healthy
  0   73 70              75                  36     9      17
  1   23 31              28                   5     0       0
[1] "===================================="
[1] "race_AmericanIndianOrAlaskaNative"
   
    MISC  KD viral_infection bacterial_infection other Healthy
  0   96 100             103                  41     9      17
  1    0   1               0                   0     0       0
[1] "===================================="
[1] "race_Asian"
   
    M

,PTID,inflam_cat,age,origin,eth_HispanicOrLatino,eth_NotHispanicOrLatino,race_HispanicOrLatino,race_AmericanIndianOrAlaskaNative,race_Asian,race_BlackOrAA,⋯,icu_yes,icu_no,oxygen_NasalCannula,oxygen_FaceMask,oxygen_HFNC,oxygen_BiPap,oxygen_Ventilator,oxygen_ECMO,oxygen_CPAP,oxygen_None
,<chr>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,153000,KD,4.744907,UCSD,0,1,0,0,0,0,⋯,0,1,NA,NA,NA,NA,NA,NA,NA,NA
2,153002,KD,8.800000,UCSD,0,1,0,0,0,0,⋯,0,1,NA,NA,NA,NA,NA,NA,NA,NA
3,153027,KD,2.900000,UCSD,0,1,0,0,0,0,⋯,0,1,NA,NA,NA,NA,NA,NA,NA,NA
4,153034,KD,2.000000,UCSD,0,1,0,0,0,0,⋯,0,1,NA,NA,NA,NA,NA,NA,NA,NA
5,153036,KD,1.741309,UCSD,1,0,1,0,0,0,⋯,0,1,NA,NA,NA,NA,NA,NA,NA,NA
6,153047,KD,2.428409,UCSD,1,0,1,0,0,0,⋯,0,1,NA,NA,NA,NA,NA,NA,NA,NA


---
## Supp Fig 1

In [3]:
meta %>% filter(grepl("238",sample_id)) %>% head()
all_map_stats %>% filter(grepl("238",sample_id)) %>% head()

,PTID,sample_id,diagnosis,origin,severity,date_dif,age,inflam_cat,Xsample_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
1,4561,cfrna_kd_238.trimmed,viral_infection,UCSD,adenovirus,0,5.659254,viral_infection,cfrna_kd_238.trimmed


ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'head': object 'all_map_stats' not found


In [31]:
meta %>% colnames()
# meta %>% select(PTID, Xsample_id, inflam_cat) %>% mutate(Xsample_id = gsub("_trimm",".trimm",Xsample_id)) %>% filter(grepl("176",Xsample_id))
# all_map_stats %>% filter(grepl("176",sample_id))

[1] "PTID"       "sample_id"  "diagnosis"  "origin"     "severity"  
[6] "date_dif"   "age"        "inflam_cat" "Xsample_id"

In [42]:
map_stats1 = read.delim("/workdir/cfrna/alignment/takara_human_V2/output/prevail/prevail_mapping_stats.tsv")
map_stats2 = read.delim("/workdir/cjl332/radysChildrens_ucsd/0_ALIGN/takara_human_V2/output/KD/KD_mapping_stats.tsv")
map_stats3 = read.delim("/workdir/cjl332/prevail/PAPER2/ALIGN/takara_human_V2/output/ped_inflam_20240103_TRIMMED/ped_inflam_20240103_TRIMMED_mapping_stats.tsv")

all_map_stats <- do.call(rbind, list(map_stats1,map_stats2,map_stats3)) %>% unique()


mdf_all <- merge(meta %>% select(PTID,origin, Xsample_id, inflam_cat) %>% mutate(Xsample_id = gsub("-trimm",".trimm",Xsample_id)), 
                        all_map_stats %>% mutate(sample_id = gsub("-trimm",".trimm",sample_id)),
                        by.x = "Xsample_id", by.y = "sample_id", all.x = TRUE) %>% 
            mutate(GRP = " ")

In [103]:
####################
### MAKE QC PLOTS
####################

WIDTH= 2
HEIGHT= 1.3

XVAR = "GRP"
XVAR = "origin"

###------------------------------------------------

VAR = "reads_all"

var_box <- mdf_all %>% 
    mutate(origin = factor(origin, levels = c("UCSD","EMORY","CNH","UCSF"))) %>% 
    ggplot(aes(x=.data[[XVAR]], y=.data[[VAR]]))+
    geom_boxplot(outlier.shape = NA, color = "black", alpha = 0.75)+
    geom_jitter(width = 0.2, height = 0, color = "black", size = .75, alpha = 0.5, stroke=NA)+
    theme_cfrna_print()+
    labs(y = gsub("_"," ",VAR))+
    theme(axis.title.x = element_blank())

pdf(file=paste0("figure_plots/SUPP_",VAR,".box.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
var_box
dev.off()

###------------------------------------------------

VAR = "intron_exon_ratio"

var_box <- mdf_all %>% 
    mutate(origin = factor(origin, levels = c("UCSD","EMORY","CNH","UCSF"))) %>% 
    ggplot(aes(x=.data[[XVAR]], y=.data[[VAR]]))+
    geom_boxplot(outlier.shape = NA, color = "black", alpha = 0.75)+
    geom_jitter(width = 0.2, height = 0, color = "black", size = .75, alpha = 0.5, stroke=NA)+
    theme_cfrna_print()+
    labs(y = gsub("_"," ",VAR))+
    theme(axis.title.x = element_blank())

pdf(file=paste0("figure_plots/SUPP_",VAR,".box.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
var_box
dev.off()

###------------------------------------------------

VAR = "ftcount"

var_box <- mdf_all %>% 
    mutate(origin = factor(origin, levels = c("UCSD","EMORY","CNH","UCSF"))) %>% 
    ggplot(aes(x=.data[[XVAR]], y=.data[[VAR]]))+
    geom_boxplot(outlier.shape = NA, color = "black", alpha = 0.75)+
    geom_jitter(width = 0.2, height = 0, color = "black", size = .75, alpha = 0.5, stroke=NA)+
    theme_cfrna_print()+
    labs(y = gsub("_"," ",VAR))+
    theme(axis.title.x = element_blank())

pdf(file=paste0("figure_plots/SUPP_",VAR,".box.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
var_box
dev.off()

###------------------------------------------------

# WIDTH= 1.8
# HEIGHT= 1.3

VAR = "bias53"

var_box <- mdf_all %>% 
    mutate(origin = factor(origin, levels = c("UCSD","EMORY","CNH","UCSF"))) %>% 
    ggplot(aes(x=.data[[XVAR]], y=.data[[VAR]]))+
    geom_boxplot(outlier.shape = NA, color = "black", alpha = 0.75)+
    geom_jitter(width = 0.2, height = 0, color = "black", size = .75, alpha = 0.5, stroke=NA)+
    theme_cfrna_print()+
    labs(y = gsub("_"," ",VAR))+
    theme(axis.title.x = element_blank())

pdf(file=paste0("figure_plots/SUPP_",VAR,".box.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
var_box
dev.off()


png 
  2

png 
  2

png 
  2

png 
  2

In [12]:
####################
### BATCH EFFECTS
####################

suppressMessages(library(DESeq2))
library(matrixStats)

VAR = "BATCH"

WIDTH= 3
HEIGHT= 3



counts <- read.csv("./counts_all.csv", row.names = 1)
# counts %>% head()

## run VST on data
counts_train_vst <- read.csv("./counts_train_vst.7030.csv", row.names=1)
counts_test_vst <- read.csv("./counts_test_vst.7030.csv", row.names=1)
counts_vst <- merge(counts_train_vst,counts_test_vst, by=0) %>% column_to_rownames(var = "Row.names") 

###--------------------------
### gene variance
###--------------------------

## get gene variance
gene_vars <- matrixStats::rowVars(as.matrix(counts_vst)) %>% data.frame() %>% dplyr::rename(.,vars = `.`) %>% arrange(desc(vars))
gene_vars$rank <- 1:nrow(gene_vars)

all_top_genes <- gene_vars %>% filter(rank <= 500) %>% rownames()

counts_vst_top <- counts_vst[all_top_genes,]


# perform a PCA on the data in assay(x) for the selected genes
pcaData <- prcomp(t(counts_vst_top),scale=TRUE)
pcaData_sum <- summary(pcaData)

percentVar <- round(100 * pcaData_sum$importance[2,],2)           ### round percent variance explained
PCsdata <- pcaData$x %>% data.frame()

PCsdata <- merge(meta, PCsdata, by.x="Xsample_id", by.y=0, all.x=TRUE)

PCsdata %>% head()
### plot PCA
options(repr.plot.width = 5 ,repr.plot.height = 5)
PC1_PC2_plt <- PCsdata %>% 
  ggplot( aes(x=PC1, y=PC2, color = origin)) +
  geom_point(size=1) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) +
  theme_cfrna_print()+
  scale_color_manual(values = FILL_KEY_ORIGIN)

pdf(file=paste0("figure_plots/SUPP_",VAR,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
PC1_PC2_plt
dev.off()


,Xsample_id,PTID,sample_id,diagnosis,origin,severity,date_dif,age,inflam_cat,PC1,⋯,PC335,PC336,PC337,PC338,PC339,PC340,PC341,PC342,PC343,PC344
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,cfrna_kd_1,4551,cfrna_kd_1,FC,UCSD,ALT normal,0,4.6682455,viral_infection,7.898717,⋯,-0.045155370,-0.145424498,0.085980678,0.0135520946,0.006897340,-0.04693548,-0.02571148,-0.09163499,-0.03008548,-2.109424e-15
2,cfrna_kd_10,184023,cfrna_kd_10,FC,UCSD,ALT normal,0,5.4072523,viral_infection,10.412708,⋯,0.042771406,0.197315371,0.025108174,-0.0407237087,0.107996697,-0.14602469,0.02892713,-0.01217700,-0.06583591,-3.469447e-15
3,cfrna_kd_100,4470,cfrna_kd_100,FC,UCSD,ALT high,0,5.8784004,viral_infection,8.555557,⋯,0.097000303,0.001868556,-0.025274586,-0.0009710211,0.016480858,-0.08770269,0.01051495,-0.05098198,-0.02129128,-1.970646e-15
4,cfrna_kd_101,4567,cfrna_kd_101,FC,UCSD,ALT high,0,6.5600252,viral_infection,24.760316,⋯,-0.003697227,-0.112374391,0.004431854,0.0731317556,0.110602755,-0.08123829,-0.09420487,-0.01733782,-0.02027187,-9.436896e-16
5,cfrna_kd_102,4523,cfrna_kd_102,FC,UCSD,ALT high,0,0.9035093,viral_infection,24.714397,⋯,0.039469305,-0.164156130,-0.020667384,-0.0344189025,0.000660851,0.01407741,0.07495208,0.04178697,0.02550578,-4.871104e-15
6,cfrna_kd_105,204112,cfrna_kd_105,FC,UCSD,ALT high,0,2.1575848,viral_infection,24.740716,⋯,-0.037303531,-0.118633082,-0.033003619,-0.0976447909,-0.009080030,0.06419100,0.06701165,0.04210526,-0.07596689,-3.108624e-15


Warning message:
“Removed 26 rows containing missing values (`geom_point()`).”


png 
  2